# Sentinel Hub Process API

In this example notebook we show how to use Sentinel Hub Process API to download satellite imagery. We describe how to use various parameters and configurations to obtain either processed products or raw band data. For more information about the service please check the [official service documentation](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process.html).

###  Imports 

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import datetime
import os
import getpass

import matplotlib.pyplot as plt
import numpy as np

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

# The following is not a package. It is a file utils.py which should be in the same folder as this notebook.
from utils import plot_image

In [3]:
# Only run this cell if you have not created a configuration.

config = SHConfig()
config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")

In [ ]:
# config = SHConfig("profile_name")

### Setting area of interest

In [ ]:
from shapely.geometry import Polygon
from constants import polygons_europe

def get_bbox_from_polygons(polygons):
    """
    Calcule la bounding box d'une liste de polygones.

    Args:
        polygons (list): Liste de polygones, chaque polygone étant une liste de coordonnées [(lon, lat), ...]

    Returns:
        tuple: Bounding box sous la forme (min_lon, min_lat, max_lon, max_lat)
    """
    min_lon, min_lat = float('inf'), float('inf')
    max_lon, max_lat = float('-inf'), float('-inf')

    for poly in polygons:
        # Créer un objet Polygon à partir des coordonnées du polygone
        shapely_poly = Polygon(poly)

        # Obtenir les coordonnées de la bounding box du polygone
        minx, miny, maxx, maxy = shapely_poly.bounds

        # Mettre à jour la bbox globale
        min_lon = min(min_lon, minx)
        min_lat = min(min_lat, miny)
        max_lon = max(max_lon, maxx)
        max_lat = max(max_lat, maxy)

    return (min_lon, min_lat, max_lon, max_lat)

# Exemple d'utilisation avec une liste de polygones
polygons = polygons_europe
bbox = get_bbox_from_polygons(polygons)
print("Bounding Box Europe:", bbox)


In [5]:
from shapely.geometry import Polygon, box
polygon_europe = Polygon(polygons[0])

In [ ]:
polygon_europe

In [ ]:
def divide_bbox(bbox, n_parts=50000):
    """
    Divise une bbox en une grille de n_parts sous-bbox.

    Args:
        bbox (tuple): La bounding box sous la forme (min_lon, min_lat, max_lon, max_lat).
        n_parts (int): Nombre approximatif de divisions souhaitées.

    Returns:
        list: Liste des sous-bounding boxes sous la forme [(min_lon, min_lat, max_lon, max_lat), ...].
    """
    min_lon, min_lat, max_lon, max_lat = bbox

    # Calcul de la largeur et hauteur totale de la bbox
    bbox_width = max_lon - min_lon
    bbox_height = max_lat - min_lat

    # Calculer un rapport largeur/hauteur pour une répartition équilibrée
    aspect_ratio = bbox_width / bbox_height

    # Déterminer le nombre optimal de lignes et colonnes pour obtenir environ n_parts cellules
    cols = int((n_parts * aspect_ratio) ** 0.5)
    rows = int(n_parts / cols)

    # Ajuster si nécessaire
    if cols * rows < n_parts:
        cols += 1

    # Calculer la taille d'une cellule
    cell_width = bbox_width / cols
    cell_height = bbox_height / rows

    # Générer les sous-bbox
    sub_bboxes = []
    for row in range(rows):
        for col in range(cols):
            sub_min_lon = min_lon + col * cell_width
            sub_max_lon = sub_min_lon + cell_width
            sub_min_lat = min_lat + row * cell_height
            sub_max_lat = sub_min_lat + cell_height

            # Ajouter la sous-bbox à la liste
            sub_bboxes.append((sub_min_lon, sub_min_lat, sub_max_lon, sub_max_lat))

    return sub_bboxes

# Diviser la bbox europe pour avoir des images de taille < à 2500 pixels pour une résolution de 10m :
sub_bboxes = divide_bbox(bbox, n_parts=50000)

# Affichage du résultat
print(f"Nombre de sous-bounding boxes : {len(sub_bboxes)}")
print("Exemple de sous-bbox :", sub_bboxes[:5])  # Affiche les 5 premières


In [8]:
from shapely.geometry import Polygon, box

sub_bboxes_included = []
for ss_box in sub_bboxes:
    ss_bbox = box(*ss_box)
    intersection = polygon_europe.intersection(ss_bbox)

    # Calcul des aires
    intersection_area = intersection.area
    bbox_area = ss_bbox.area

    # Vérification si au moins 20% de la bbox est incluse
    percentage_included = (intersection_area / bbox_area) * 100
    is_included = percentage_included >= 20
    if is_included:
        sub_bboxes_included.append(ss_box)

In [ ]:
len(sub_bboxes_included)

In [ ]:
resolution = 10
zone_bbox = BBox(bbox=list(sub_bboxes_included[5000]), crs=CRS.WGS84)
zone_size = bbox_to_dimensions(zone_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {zone_size} pixels")

evalscript_all_bands = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"],
                units: "DN"
            }],
            output: {
                bands: 13,
                sampleType: "INT16"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B01,
                sample.B02,
                sample.B03,
                sample.B04,
                sample.B05,
                sample.B06,
                sample.B07,
                sample.B08,
                sample.B8A,
                sample.B09,
                sample.B10,
                sample.B11,
                sample.B12];
    }
"""

In [13]:
def transform_string(input_string):
    # Remplace les caractères selon les règles spécifiées
    result = input_string.replace('(', '').replace(')', '').replace(', ', '_').replace('.', ',')
    return result
filename = transform_string(str(sub_bboxes_included[5000]))

In [14]:
request_all_bands = SentinelHubRequest(
    data_folder="europe_2023/"+filename,
    evalscript=evalscript_all_bands,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from(
                "s2l1c", service_url=config.sh_base_url
            ),
            time_interval=("2023-01-01", "2023-01-30"),
            mosaicking_order=MosaickingOrder.LEAST_CC,
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=zone_bbox,
    size=zone_size,
    config=config,
)

In [ ]:
%%time
all_bands_img = request_all_bands.get_data(save_data=True)

In [ ]:
# True color image
plot_image(all_bands_img[0][:, :, [3, 2, 1]], factor=3.5 / 1e4, clip_range=(0, 1))

In [ ]:
%%time
# try to re-download the data
all_bands_img_from_disk = request_all_bands.get_data()

In [ ]:
%%time
# force the redownload
all_bands_img_redownload = request_all_bands.get_data(redownload=True)

### Example 4.1: Save downloaded data directly to disk

The `get_data` method returns a list of numpy arrays and can save the downloaded data to disk, as we have seen in the previous example. Sometimes it is convenient to just save the data directly to disk. You can do that by using `save_data` method instead.

In [ ]:
%%time
request_all_bands.save_data()

In [ ]:
print(
    "The output directory has been created and a tiff file with all 13 bands was saved into the following structure:\n"
)

for folder, _, filenames in os.walk(request_all_bands.data_folder):
    for filename in filenames:
        print(os.path.join(folder, filename))

## Example 5: Other Data Collections

The `sentinelhub-py` package supports various data collections. The example below is shown for one of them, but the process is the same for all of them.


<div class="alert alert-info">

<b>Note:</b>
    
For more examples and information check the [documentation about Sentinel Hub data collections](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Data.html).
</div>

In [ ]:
print("Supported DataCollections:\n")
for collection in DataCollection.get_available_collections():
    print(collection)

For this example let's download the digital elevation model data (DEM). The process is similar as before, we just provide the evalscript and create the request. More data on the `DEM` data collection is available [here](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Data/DEM.html). `DEM` values are in meters and can be negative for areas which lie below sea level, so it is recommended to set the output format in your evalscript to `FLOAT32`.

In [ ]:
evalscript_dem = """
//VERSION=3
function setup() {
  return {
    input: ["DEM"],
    output:{
      id: "default",
      bands: 1,
      sampleType: SampleType.FLOAT32
    }
  }
}

function evaluatePixel(sample) {
  return [sample.DEM]
}
"""

In [ ]:
dem_request = SentinelHubRequest(
    evalscript=evalscript_dem,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.DEM.define_from(
                "dem", service_url=config.sh_base_url
            ),
            time_interval=("2020-06-12", "2020-06-13"),
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=zone_bbox,
    size=zone_size,
    config=config,
)

In [ ]:
dem_data = dem_request.get_data()

In [ ]:
# Plot DEM map
# vmin = 0; cutoff at sea level (0 m)
# vmax = 120; cutoff at high values (120 m)
plot_image(dem_data[0], factor=1.0, cmap=plt.cm.Greys_r, vmin=0, vmax=120)

## Example 6 : Multi-response request type

Process API enables downloading multiple files in one response, packed together in a TAR archive.

We will get the same image as before, download in the form of digital numbers (DN) as a UINT16 TIFF file. Along with the image we will download the `inputMetadata` which contains the normalization factor value in a JSON format. 

After the download we will be able to convert the `INT16` digital numbers to get the `FLOAT32` reflectances.

In [ ]:
evalscript = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"],
                units: "DN"
            }],
            output: {
                bands: 3,
                sampleType: "INT16"
            }
        };
    }

    function updateOutputMetadata(scenes, inputMetadata, outputMetadata) {
        outputMetadata.userData = { "norm_factor":  inputMetadata.normalizationFactor }
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

request_multitype = SentinelHubRequest(
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from(
                "s2l1c", service_url=config.sh_base_url
            ),
            time_interval=("2020-06-01", "2020-06-30"),
            mosaicking_order=MosaickingOrder.LEAST_CC,
        )
    ],
    responses=[
        SentinelHubRequest.output_response("default", MimeType.TIFF),
        SentinelHubRequest.output_response("userdata", MimeType.JSON),
    ],
    bbox=zone_bbox,
    size=zone_size,
    config=config,
)

In [ ]:
# print out information
multi_data = request_multitype.get_data()[0]
multi_data.keys()

In [ ]:
# normalize image
img = multi_data["default.tif"]
norm_factor = multi_data["userdata.json"]["norm_factor"]

img_float32 = img * norm_factor

In [ ]:
plot_image(img_float32, factor=3.5, clip_range=(0, 1))

## Example 7 : Raw dictionary request

All requests so far were built with some helper functions. We can also construct a raw dictionary as defined in the [API Reference](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/ApiReference.html), without these helper functions, so we have full control over building the request body.

In [ ]:
request_raw_dict = {
    "input": {
        "bounds": {
            "properties": {"crs": zone_bbox.crs.opengis_string},
            "bbox": list(zone_bbox),
        },
        "data": [
            {
                "type": "S2L1C",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-06-01T00:00:00Z",
                        "to": "2020-06-30T00:00:00Z",
                    },
                    "mosaickingOrder": "leastCC",
                },
            }
        ],
    },
    "output": {
        "width": zone_size[0],
        "height": zone_size[1],
        "responses": [
            {"identifier": "default", "format": {"type": MimeType.TIFF.get_string()}}
        ],
    },
    "evalscript": evalscript_true_color,
}

In [ ]:
# create request
download_request = DownloadRequest(
    request_type="POST",
    url="https://sh.dataspace.copernicus.eu/api/v1/process",
    post_values=request_raw_dict,
    data_type=MimeType.TIFF,
    headers={"content-type": "application/json"},
    use_session=True,
)

# execute request
client = SentinelHubDownloadClient(config=config)
img = client.download(download_request)

In [ ]:
plot_image(img, factor=3.5 / 255, clip_range=(0, 1))

## Example 8 : Multiple timestamps data

It is possible to construct some logic in order to return data for multiple timestamps. By defining the `time_interval` parameter and some logic of splitting it, it is possible to create an SH reques per each "time slot" and then download the data from all the requests with the `SentinelHubDownloadClient` in `sentinelhub-py`. In this example we will create least cloudy monthly images for the year 2019.

However, this is already a functionality built on top of this SH API package. We have extended the support for such usage in our package [eo-learn](https://github.com/sentinel-hub/eo-learn). We recommend to use `eo-learn` for more complex cases where you need multiple timestamps or high-resolution data for larger areas.

In [ ]:
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2019, 12, 31)
n_chunks = 13
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Monthly time windows:\n")
for slot in slots:
    print(slot)

In [ ]:
def get_true_color_request(time_interval):
    return SentinelHubRequest(
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C.define_from(
                    "s2l1c", service_url=config.sh_base_url
                ),
                time_interval=time_interval,
                mosaicking_order=MosaickingOrder.LEAST_CC,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=zone_bbox,
        size=zone_size,
        config=config,
    )

In [ ]:
# create a list of requests
list_of_requests = [get_true_color_request(slot) for slot in slots]
list_of_requests = [request.download_list[0] for request in list_of_requests]

# download data with multiple threads
data = SentinelHubDownloadClient(config=config).download(
    list_of_requests, max_threads=5
)

In [ ]:
# some stuff for pretty plots
ncols = 4
nrows = 3
aspect_ratio = zone_size[0] / zone_size[1]
subplot_kw = {"xticks": [], "yticks": [], "frame_on": False}

fig, axs = plt.subplots(
    ncols=ncols,
    nrows=nrows,
    figsize=(5 * ncols * aspect_ratio, 5 * nrows),
    subplot_kw=subplot_kw,
)

for idx, image in enumerate(data):
    ax = axs[idx // ncols][idx % ncols]
    ax.imshow(np.clip(image * 2.5 / 255, 0, 1))
    ax.set_title(f"{slots[idx][0]}  -  {slots[idx][1]}", fontsize=10)

plt.tight_layout()